In [ ]:
from os import listdir
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def preprocessing(text_file):
    tokenizer = nltk.RegexpTokenizer(r"\w+")    # remove punctuations
    tokens = tokenizer.tokenize(text_file)    # token the text

    tokens = [token.lower() for token in tokens]    # lower case
    tokens = [word for word in tokens if not word in stop_words]    # remove stop words

    stemmer = PorterStemmer()    #stemming tokens
    tokens = [stemmer.stem(word) for word in tokens]
    
    lemmatizer = WordNetLemmatizer()   #lemmatizing tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

In [ ]:
# get a list of all files in the directory
def getFilesList(folder):
    dir_list = listdir(folder)
    files = {}
    for i in dir_list:
        path = folder+i+"/"
        diri = listdir(path)
        # print(len(diri))
        # print(diri)
        files[i] = []
        for j in diri:
            filepath = path+j
            files[i].append(filepath)
            # print(filepath)
            # break

    # for i in files:
    #     print(len(files[i]))
    return files

In [ ]:
folderpath = "20_newsgroups/"

def files_read_preprocess(folderpath):
    flist = getFilesList(folderpath)
    # textFolderwise = {}
    alltextfromfiles = []
    textlabels = []
    for folder in flist:
        # textFolderwise[folder] = []
        for file in flist[folder]:
            # print(file)
            text = open(file,'r',encoding='ISO-8859-1').read()
            tokens = preprocessing(text)
            # textFolderwise[folder].append(tokens)
            alltextfromfiles.append(tokens)
            textlabels.append(folder)

    print(len(alltextfromfiles))
    print(len(textlabels))

    dataset = pd.DataFrame([alltextfromfiles,textlabels]).T
    dataset.to_pickle("q3_preprocessed")

In [ ]:
# dataset = files_read_preprocess(folderpath)
dataset = pd.read_pickle("q3_preprocessed")
print(dataset)

y = dataset.pop(1)
X = dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                                                      0                 1
0     [path, cantaloup, srv, c, cmu, edu, da, news, ...  rec.sport.hockey
1     [newsgroup, rec, sport, hockey, path, cantalou...  rec.sport.hockey
2     [path, cantaloup, srv, c, cmu, edu, da, news, ...  rec.sport.hockey
3     [newsgroup, rec, sport, hockey, path, cantalou...  rec.sport.hockey
4     [newsgroup, rec, sport, hockey, path, cantalou...  rec.sport.hockey
...                                                 ...               ...
4995  [xref, cantaloup, srv, c, cmu, edu, sci, med, ...           sci.med
4996  [path, cantaloup, srv, c, cmu, edu, rochest, u...           sci.med
4997  [xref, cantaloup, srv, c, cmu, edu, sci, med, ...           sci.med
4998  [path, cantaloup, srv, c, cmu, edu, crabappl, ...           sci.med
4999  [xref, cantaloup, srv, c, cmu, edu, sci, med, ...           sci.med

[5000 rows x 2 columns]


In [ ]:
def termfrequency_tf(labels,mvar):
    class_frequency = {}
    class_count = {}
    counter = 0
    for i in labels:
        current_count = len(Counter(m[i]))
        class_count[i] = current_count
        counter += current_count
        ll = Counter(m[i])
        for j in ll:
            class_frequency[i, j] = ll[j]
        
    return class_frequency,class_count

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5be6439c-c857-493e-9eef-529c4c49e2bf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>